In [0]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint

# Load and Prepare Data

#### Format of the data
- last column of the data frame must contain the label and it must also be called "label"
- there should be no missing values in the data frame

In [0]:
df = pd.read_csv("/content/train_modified.csv")
# df = df.drop("Id", axis=1)


In [0]:
df.head()

,Timestamp,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,P201,P202,P203,P204,P205,P206,DPIT301,FIT301,LIT301,MV301,MV302,MV303,MV304,P301,P302,AIT401,AIT402,FIT401,LIT401,P401,P402,P403,P404,UV401,AIT501,AIT502,AIT503,AIT504,FIT501,FIT502,FIT503,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603,Normal/Attack
0,22/12/2015 4:00:00 PM,2.470294,261.5804,2,2,1,244.3284,8.19008,306.101,2.471278,2,1,1,2,1.0,2.0,1.0,20.79839,2.235275,327.4401,1.0,2.0,1.0,1.0,2.0,1.0,0.0,171.0587,0.0,238.0544,1.0,1.0,1.0,1.0,1.0,7.308575,176.8265,263.6504,12.68905,0.001666,0.001409,0.001664,0.0,1.0,1.0,10.02948,0.0,4.277749,0.000256,1.0,1.0,1.0,Normal
1,22/12/2015 4:00:01 PM,2.457163,261.1879,2,2,1,244.3284,8.19008,306.101,2.468587,2,1,1,2,1.0,2.0,1.0,20.79839,2.234507,327.4401,1.0,2.0,1.0,1.0,2.0,1.0,0.0,171.0587,0.0,238.1312,1.0,1.0,1.0,1.0,1.0,7.308575,176.8008,263.6504,12.68905,0.001666,0.001409,0.001664,0.0,1.0,1.0,10.02948,0.0,4.277749,0.000256,1.0,1.0,1.0,Normal
2,22/12/2015 4:00:02 PM,2.439548,260.9131,2,2,1,244.3284,8.19008,306.101,2.467305,2,1,1,2,1.0,2.0,1.0,20.84320,2.233354,327.4401,1.0,2.0,1.0,1.0,2.0,1.0,0.0,171.0587,0.0,238.0544,1.0,1.0,1.0,1.0,1.0,7.308575,176.8008,263.6504,12.68905,0.001666,0.001409,0.001664,0.0,1.0,1.0,10.02948,0.0,4.277749,0.000256,1.0,1.0,1.0,Normal
3,22/12/2015 4:00:03 PM,2.428338,260.2850,2,2,1,244.3284,8.19008,306.101,2.466536,2,1,1,2,1.0,2.0,1.0,20.84320,2.233354,327.2799,1.0,2.0,1.0,1.0,2.0,1.0,0.0,171.0587,0.0,238.2081,1.0,1.0,1.0,1.0,1.0,7.308575,176.8008,263.6504,12.68905,0.001666,0.001409,0.001664,0.0,1.0,1.0,10.02948,0.0,4.277749,0.000256,1.0,1.0,1.0,Normal
4,22/12/2015 4:00:04 PM,2.424815,259.8925,2,2,1,244.4245,8.19008,306.101,2.466536,2,1,1,2,1.0,2.0,1.0,20.84320,2.233354,327.1597,1.0,2.0,1.0,1.0,2.0,1.0,0.0,171.0587,0.0,238.4389,1.0,1.0,1.0,1.0,1.0,7.308575,176.8008,263.6504,12.68905,0.001666,0.001409,0.001664,0.0,1.0,1.0,10.02948,0.0,4.277749,0.000256,1.0,1.0,1.0,Normal


# Train-Test-Split

In [0]:
import pandas as pd
import numpy as np
from pprint import pprint
dataset = pd.read_csv('/content/train_modified.csv')

In [0]:
def cal_entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    cal_entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return cal_entropy

In [0]:
def InfoGain(data,split_attribute_name,target_name="class"):
  
    #Calculate the entropy of the total dataset
    total_entropy = entropy(data[target_name])
    
    ##Calculate the entropy of the dataset
    
    #Calculate the values and the corresponding counts for the split attribute 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    
    #Calculate the weighted entropy
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    #Calculate the information gain
    Information_Gain = total_entropy - Weighted_Entropy
    return Information_Gain

In [0]:
def ID3(data,originaldata,features,target_attribute_name="class",parent_node_class = None):

    #Define the stopping criteria --> If one of this is satisfied, we want to return a leaf node#
    
    #If all target_values have the same value, return this value
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    #If the dataset is empty, return the mode target feature value in the original dataset
    elif len(data)==0:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
     
    elif len(features) ==0:
        return parent_node_class
    
    #If none of the above holds true, grow the tree!
    
    else:
        #Set the default value for this node --> The mode target feature value of the current node
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        
        #Create the tree structure. The root gets the name of the feature (best_feature) with the maximum information
        #gain in the first run
        tree = {best_feature:{}}
        
        
        #Remove the feature with the best inforamtion gain from the feature space
        features = [i for i in features if i != best_feature]
        
        #Grow a branch under the root node for each possible value of the root node feature
        
        for value in np.unique(data[best_feature]):
            value = value
            #Split the dataset along the value of the feature with the largest information gain and therwith create sub_datasets
            sub_data = data.where(data[best_feature] == value).dropna()
            
            #Call the ID3 algorithm for each of those sub_datasets with the new parameters --> Here the recursion comes in!
            subtree = ID3(sub_data,dataset,features,target_attribute_name,parent_node_class)
            
            #Add the sub tree, grown from the sub_dataset to the tree under the root node
            tree[best_feature][value] = subtree
            
        return(tree)    
                
    
    
def predict(query,tree,default = 1):

    for key in list(query.keys()):
        if key in list(tree.keys()):
            #2.
            try:
                result = tree[key][query[key]] 
            except:
                return default
  
            #3.
            result = tree[key][query[key]]
            #4.
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result
        
        

def train_test_split(dataset):
    training_data = dataset.iloc[:80].reset_index(drop=True)#We drop the index respectively relabel the index
    #starting form 0, because we do not want to run into errors regarding the row labels / indexes
    testing_data = dataset.iloc[80:].reset_index(drop=True)
    return training_data,testing_data
training_data = train_test_split(dataset)[0]
testing_data = train_test_split(dataset)[1] 
def test(data,tree):
    #Create new query instances by simply removing the target feature column from the original dataset and 
    #convert it to a dictionary
    queries = data.iloc[:,:-1].to_dict(orient = "records")
    
    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    predicted = pd.DataFrame(columns=["predicted"]) 
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The prediction accuracy is: ',(np.sum(predicted["predicted"] == data["class"])/len(data))*100,'%')

tree = ID3(training_data,training_data,training_data.columns[:-1])
pprint(tree)
test(testing_data,tree)

KeyError: ignored

In [0]:
random.seed(0)
train_df, test_df = train_test_split(df, test_size=20)

In [0]:
kf=train_df.loc[:,'MV201':'P206'].values

In [0]:
kf

array([[2, 1, 1, ..., 1, 2, 1],
       [2, 1, 1, ..., 1, 2, 1],
       [2, 1, 1, ..., 1, 2, 1],
       ...,
       [2, 1, 1, ..., 1, 2, 1],
       [2, 1, 1, ..., 1, 2, 1],
       [2, 1, 1, ..., 1, 2, 1]])

In [0]:
unique_classes, counts_unique_classes = np.unique(kf, return_counts=True)

In [0]:
unique_classes, counts_unique_classes

(array([1, 2]), array([2406392, 1071068]))

In [0]:
print(counts_unique_classes)

[2406392 1071068]


# Helper Functions

In [0]:
data = train_df.loc[:,"FIT101":"P603"].values
data[:5]

In [0]:
len(data[0])

51

#Probability

In [0]:
def gini(data):
  label_column = data[:-1]
  unique_classes=np.unique(label_column)
  

### Check Purity of data

In [0]:
def check_purity(data):
    
    label_column = data[:-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [0]:
def classify_data(data):
    
    label_column = data[:,-1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)
    print(np.asarray((unique_classes, counts_unique_classes)))

    index = counts_unique_classes.argmax()
    print(index)
    classification = unique_classes[index]
    
    return classification

In [0]:
data.shape

(496780, 51)

In [0]:
for i in range(data.shape[1]):
  unique, counts = np.unique(data[:,i], return_counts=True)
  k=dict(zip(unique, counts))
  print( k)

In [0]:
np.unique(kf[:,1])

array([1])

### check for splits

In [0]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        type_of_feature = FEATURE_TYPES[column_index]
        if type_of_feature == "continuous":
            potential_splits[column_index] = []
            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[column_index].append(potential_split)
        
        # feature is categorical
        # (there need to be at least 2 unique values, otherwise in the
        # split_data function data_below would contain all data points
        # and data_above would be empty)
        elif len(unique_values) > 1:
            potential_splits[column_index] = unique_values
    
    return potential_splits

In [0]:
kf

array([2, 1, 1, 2, 1, 2, 1])

In [0]:
get_potential_splits(kf)

{0: [1.5], 1: [], 3: array([1, 2]), 5: [1.5]}

### Splitting the Data

In [0]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

### find the cross entropy

In [0]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

### find the overall entropy

In [0]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [0]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

# Decision Tree Algorithm

### Representation of the Decision Tree

### Determine Type of Feature

In [0]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

### Algorithm

In [0]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [0]:
k_train=train_df.loc[:,"FIT101":"P603"]

In [0]:
tree = decision_tree_algorithm(k_train, max_depth=3)
pprint(tree)